In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My\ Drive/nlp/sentiment_analysis

/content/drive/My Drive/nlp/sentiment_analysis


In [3]:
! pip install transformers
! pip install torchsummary

     |████████████████████████████████| 1.3MB 9.8MB/s eta 0:00:01
     |████████████████████████████████| 2.9MB 28.0MB/s 
     |████████████████████████████████| 890kB 47.8MB/s 
     |████████████████████████████████| 1.1MB 44.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1020f2788cdca2b62ce92e056da9ad12f22c53ac92f08d9a8c321fa391da2f6f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [1]:
import os 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pathlib import Path

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AdamW
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
import torch
from tqdm.notebook import tqdm
from torchsummary import summary

In [23]:
# GPU 사용
device = torch.device("cuda")

In [3]:
class SentimentReviewDataset(Dataset):
  
  def __init__(self, dataset):
    self.dataset = dataset
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:2].values
    text = row[1]
    y = row[0]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [28]:
def merge_review_dataset(data_dir_list):
    # naver shopping review 
    shopping_review_dataset = pd.read_csv(data_dir_list[0], sep='\t', names=['star','review'])
    shopping_review_dataset['star'] = [1 if data['star'] >= 4 else 0 for idx, data in shopping_review_dataset.iterrows()]
    shopping_review_dataset.rename(columns = {"star": "label"}, inplace=True)

    # naver movie review 
    movie_review_dataset = pd.read_csv(data_dir_list[1], sep='\t')
    movie_review_dataset = movie_review_dataset[['label','document']]
    movie_review_dataset.rename(columns={'document':'review'}, inplace=True)

    # steam game review 
    steam_review_dataset = pd.read_csv(data_dir_list[2], sep='\t', names=['label','review'])

    # merge 
    review_dataset = pd.concat([shopping_review_dataset, movie_review_dataset, steam_review_dataset], ignore_index=True)

    return review_dataset

In [29]:
data_dir_list = ['./naver_shopping_review_dataset.txt', './ratings.txt', './steam.txt']
review_dataset = merge_review_dataset(data_dir_list)
review_dataset.dropna(inplace=True)

# split train test
train_data, test_data = train_test_split(review_dataset, test_size=0.2) 

In [10]:
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [11]:
tokenizer.vocab_size

35000

In [12]:
train_dataset = SentimentReviewDataset(train_data)
test_dataset = SentimentReviewDataset(test_data)

In [13]:
train_dataset[0]

C:\Users\jaehyeong\miniconda3\envs\env_nlp\lib\site-packages\transformers\tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(tensor([    2, 11268,  7695,  4219,  8754,  4086, 21888,  4150,    18,     3,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

## Create Model

In [26]:
model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [ ]:
# # 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

In [ ]:
# model.load_state_dict(torch.load("model.pt"))
# model

In [19]:
epochs = 3
batch_size = 16

In [20]:
optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

In [ ]:
losses, accuracies

In [ ]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

In [ ]:
# 모델 저장하기
save_model_path = './_weights'
if os.path.exists(save_model_path) == False:
    os.mkdir(save_model_path)  	
torch.save(model.state_dict(), os.path.join(save_model_path, "koelectra-base-finetuned-sentiment-analysis.bin"))

## Evaluate

In [2]:
from sklearn.metrics import classification_report

In [3]:
# naver shopping review 
shopping_review_dataset = pd.read_csv('./_data/naver_shopping_review.txt', sep='\t', names=['star','review'])
shopping_review_dataset['star'] = [1 if data['star'] >= 4 else 0 for idx, data in shopping_review_dataset.iterrows()]
shopping_review_dataset.rename(columns = {"star": "label"}, inplace=True)

# # naver movie review 
# movie_review_dataset = pd.read_csv('./_data/naver_movie_ratings.txt', sep='\t')
# movie_review_dataset = movie_review_dataset[['label','document']]
# movie_review_dataset.rename(columns={'document':'review'}, inplace=True)
# movie_review_dataset.dropna(inplace=True)

# # steam game review 
# steam_review_dataset = pd.read_csv('./_data/steam_game_review.txt', sep='\t', names=['label','review'])
# steam_review_dataset.dropna(inplace=True)

In [6]:
train_data, test_data = train_test_split(shopping_review_dataset, test_size=0.2, random_state=42) 
print(len(test_data))

40000


In [10]:
## sentiment analysis
koelectra_finetuned_model_dir = os.path.join('_weights','koelectra-base-finetuned-sentiment-analysis.bin')
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator")
model.load_state_dict(torch.load(koelectra_finetuned_model_dir, map_location=torch.device('cpu')))
sentiment_classifier = pipeline('sentiment-analysis', tokenizer=tokenizer, model=model)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [11]:
y_pred = []
total_len = len(test_data)
for cnt, review in enumerate(test_data['review']):
    pred = sentiment_classifier(review)
#     print(f"{cnt} / {total_len} : {pred[0]}")
    if pred[0]['label'] == 'LABEL_1':
        y_pred.append(1)
    else:
        y_pred.append(0)

In [12]:
print(classification_report(test_data['label'], y_pred))

              precision    recall  f1-score   support

           0       0.97      0.93      0.95     19975
           1       0.93      0.97      0.95     20025

    accuracy                           0.95     40000
   macro avg       0.95      0.95      0.95     40000
weighted avg       0.95      0.95      0.95     40000

